In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [17]:
path = os.getcwd() 
files = [files for files in os.listdir(path+'/sale_data')]
all_months_data = pd.DataFrame()
for file in files:
    current_data = pd.read_csv('sale_data/' + file)
    all_months_data = pd.concat([all_months_data, current_data])
    
all_months_data.to_csv("sale_data/all_sales.csv", index=False)

In [58]:
all_data = pd.read_csv("sale_data/all_sales.csv")
all_data.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,295665,Macbook Pro Laptop,1,1700,12/30/19 00:01,"136 Church St, New York City, NY 10001"
1,295666,LG Washing Machine,1,600.0,12/29/19 07:03,"562 2nd St, New York City, NY 10001"
2,295667,USB-C Charging Cable,1,11.95,12/12/19 18:21,"277 Main St, New York City, NY 10001"
3,295668,27in FHD Monitor,1,149.99,12/22/19 15:13,"410 6th St, San Francisco, CA 94016"
4,295669,USB-C Charging Cable,1,11.95,12/18/19 12:38,"43 Hill St, Atlanta, GA 30301"


In [26]:
print(all_data.isna().sum())
print("-"*80)
nan = all_data[all_data.isna().any(axis=1)]
display(nan.head())
all_data = all_data.dropna()


Order ID            545
Product             545
Quantity Ordered    545
Price Each          545
Order Date          545
Purchase Address    545
dtype: int64
--------------------------------------------------------------------------------


,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
264,NaN,NaN,NaN,NaN,NaN,NaN
648,NaN,NaN,NaN,NaN,NaN,NaN
680,NaN,NaN,NaN,NaN,NaN,NaN
1385,NaN,NaN,NaN,NaN,NaN,NaN
1495,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 186305 entries, 0 to 186849
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Order ID          186305 non-null  object
 1   Product           186305 non-null  object
 2   Quantity Ordered  186305 non-null  object
 3   Price Each        186305 non-null  object
 4   Order Date        186305 non-null  object
 5   Purchase Address  186305 non-null  object
dtypes: object(6)
memory usage: 9.9+ MB


In [42]:
pd.to_datetime(all_data['Order Date'].str[0:8],format='%m/%d/%y')

ValueError: time data 'Order Da' does not match format '%m/%d/%y' (match)

In [36]:
all_data['Order Date'].str[0:pd.to_datetime(all_data['Order Date'].strpd.to_datetime())]

0         12/30/19 
1         12/29/19 
2         12/12/19 
3         12/22/19 
4         12/18/19 
            ...    
186845    06/07/19 
186846    06/01/19 
186847    06/22/19 
186848    06/26/19 
186849    06/25/19 
Name: Order Date, Length: 186305, dtype: object

In [44]:
all_data['Order Date'] = all_data['Order Date'].str[0:8]

In [45]:
all_data.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,295665,Macbook Pro Laptop,1,1700,12/30/19,"136 Church St, New York City, NY 10001"
1,295666,LG Washing Machine,1,600.0,12/29/19,"562 2nd St, New York City, NY 10001"
2,295667,USB-C Charging Cable,1,11.95,12/12/19,"277 Main St, New York City, NY 10001"
3,295668,27in FHD Monitor,1,149.99,12/22/19,"410 6th St, San Francisco, CA 94016"
4,295669,USB-C Charging Cable,1,11.95,12/18/19,"43 Hill St, Atlanta, GA 30301"


In [46]:
pd.to_datetime(all_data['Order Date'], format='%m/%d/%y')

ValueError: time data 'Order Da' does not match format '%m/%d/%y' (match)

In [48]:
import datetime
datetime.datetime.strptime(all_data['Order Date'], format='%m/%d/%y')

TypeError: strptime() takes no keyword arguments

In [50]:
import datetime
datetime.strptime(all_data['Order Date'], '%m/%d/%y')

AttributeError: module 'datetime' has no attribute 'strptime'

In [51]:
all_data['Order Date'],

(0         12/30/19
 1         12/29/19
 2         12/12/19
 3         12/22/19
 4         12/18/19
             ...   
 186845    06/07/19
 186846    06/01/19
 186847    06/22/19
 186848    06/26/19
 186849    06/25/19
 Name: Order Date, Length: 186305, dtype: object,)

In [55]:
all_data = all_data[all_data['Order Date'].str[0:2]!='Or']
all_data['Order Date'] = pd.to_datetime(all_data['Order Date'])
all_data

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,295665,Macbook Pro Laptop,1,1700,12/30/19 00:01,"136 Church St, New York City, NY 10001"
1,295666,LG Washing Machine,1,600.0,12/29/19 07:03,"562 2nd St, New York City, NY 10001"
2,295667,USB-C Charging Cable,1,11.95,12/12/19 18:21,"277 Main St, New York City, NY 10001"
3,295668,27in FHD Monitor,1,149.99,12/22/19 15:13,"410 6th St, San Francisco, CA 94016"
4,295669,USB-C Charging Cable,1,11.95,12/18/19 12:38,"43 Hill St, Atlanta, GA 30301"
...,...,...,...,...,...,...
186845,222905,AAA Batteries (4-pack),1,2.99,06/07/19 19:02,"795 Pine St, Boston, MA 02215"
186846,222906,27in FHD Monitor,1,149.99,06/01/19 19:29,"495 North St, New York City, NY 10001"
186847,222907,USB-C Charging Cable,1,11.95,06/22/19 18:57,"319 Ridge St, San Francisco, CA 94016"
186848,222908,USB-C Charging Cable,1,11.95,06/26/19 18:35,"916 Main St, San Francisco, CA 94016"


In [64]:
pd.to_datetime(all_data['Order Date'])

ParserError: Unknown string format: Order Date

In [60]:
all_data['Order Date'].str[0:2].unique()

array(['12', 'Or', nan, '01', '04', '05', '02', '03', '08', '09', '06',
       '11', '10', '07'], dtype=object)